In [1]:
import torch
import sys
import os.path as osp
import os
import sys
import numpy as np

sys.path.append("/afs/cern.ch/work/m/mgarciam/private/mlpf/")
from src.dataset.dataset import SimpleIterDataset
from src.utils.utils import to_filelist
from torch.utils.data import DataLoader
from tqdm import tqdm
from torch_scatter import scatter_sum
import matplotlib.pyplot as plt
import pickle
import numpy as np
import mplhep as hep


hep.style.use("CMS")
import matplotlib
matplotlib.rc('font', size=13)


/afs/cern.ch/work/m/mgarciam/private/miniconda/miniconda3/envs/graphgps0/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [23]:
import awkward as ak
outputs = ak.from_parquet("/eos/experiment/fcc/users/m/mgarciam/mlpf/condor/test_TruthTrackingGen/26/out_reco_edm4hep_GT_REC.parquet")
outputs["X_gen"][0][0,18:]

<Array [-2.15e+03, 630, -670] type='3 * float64'>

/eos/experiment/fcc/ee/datasets/DC_tracking/Pythia/gun_fakeCalo_g1/reco_gun_1000.root

In [31]:
#"/eos/experiment/fcc/ee/datasets/DC_tracking/Pythia_evaluation/Zcard/reco_Zcard_1.root"
datasets = {
    "test": "/eos/experiment/fcc/users/m/mgarciam/mlpf/condor/test_TruthTrackingGen/26/out_reco_edm4hep_REC.parquet",
    "train": "/eos/experiment/fcc/users/m/mgarciam/mlpf/condor/test_TruthTrackingGen/26/out_reco_edm4hep_REC.parquet",
}

class Args:
    def __init__(self, datasets):
        self.data_train = [datasets]
        self.data_val = [datasets]
        #self.data_train = files_train
        self.data_config = '/afs/cern.ch/work/m/mgarciam/private/mlpf/config_files/config_hits_track_v4.yaml'
        self.extra_selection = None
        self.train_val_split = 1
        self.data_fraction = 1
        self.file_fraction = 1
        self.fetch_by_files = False
        self.fetch_step = 0.1
        self.steps_per_epoch = None
        self.in_memory = False
        self.local_rank = None
        self.copy_inputs = False
        self.no_remake_weights = False
        self.batch_size = 1
        self.num_workers = 0
        self.demo = False
        self.laplace = False
        self.diffs = False
        self.class_edges = False
        self.allegro = False
        self.truth_tracking = True


args = {key: Args(value) for key, value in datasets.items()}

datas = {}
files_dict = {}
for key in datasets:
    train_range = (0, args[key].train_val_split)
    train_file_dict, train_files = to_filelist(args[key], 'val')
    train_data = SimpleIterDataset(train_file_dict, args[key].data_config, for_training=True,
                                   extra_selection=args[key].extra_selection,
                                   remake_weights=True,
                                   load_range_and_fraction=(train_range, args[key].data_fraction),
                                   file_fraction=args[key].file_fraction,
                                   fetch_by_files=args[key].fetch_by_files,
                                   fetch_step=args[key].fetch_step,
                                   infinity_mode=False,
                                   in_memory=args[key].in_memory,
                                   async_load=False,
                                   args_parse=Args(""),
                                   name='train')
    datas[key] = train_data
    files_dict[key] = train_files

['/eos/experiment/fcc/users/m/mgarciam/mlpf/condor/test_TruthTrackingGen/26/out_reco_edm4hep_REC.parquet']
['/eos/experiment/fcc/users/m/mgarciam/mlpf/condor/test_TruthTrackingGen/26/out_reco_edm4hep_REC.parquet']


In [25]:
Args(datasets.items())

In [32]:
import plotly
import plotly.graph_objects as go
import plotly.offline as pyo
from plotly.subplots import make_subplots
itera = iter(train_data)

=== Restarting DataIter train, seed=None ===
Start fetching next batch, len(filelist)=1, load_range=(0.0, 0.1)


In [33]:
g, y = next(itera)
len(y.pid), torch.unique(g.ndata["particle_number"])

Start fetching next batch, len(filelist)=1, load_range=(0.1, 0.2)
tensor([[-2149.8459,   629.8981,  -670.3370],
        [-2210.8366,  1302.8200,  -319.1441],
        [-6100.0000,  5579.2665, -1843.4112],
        [-1771.6574,  1232.1925, -1052.5925],
        [-1780.3226,  1220.0672,    38.5960],
        [-1660.4524,  1697.0923,    31.6217],
        [-1905.5631,  1041.4007,  -408.9999],
        [-6100.0000,  3957.8314,  -178.8162],
        [-1780.6062,  1388.5030,  -393.1886],
        [  732.4267,  2106.1088, -2144.0067],
        [-1922.8858,   972.2228,  -136.9274],
        [-1853.1640,  1605.7366, -1134.4022]], dtype=torch.float64)


(12, tensor([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.]))

In [34]:
y.pid

tensor([   11., -2112.,   -13.,    22.,    22.,  2212.,    22.,    13.,  2112.,
          -13.,    22., -2112.], dtype=torch.float64)

In [28]:
y.pid

tensor([   11., -2112.,   -13.,    22.,    22.,  2212.,    22.,    13.,  2112.,
          -13.,    22., -2112.], dtype=torch.float64)

In [29]:
print(g.ndata['particle_number'][g.ndata['hit_type']==1])
g.ndata['p_hits'][g.ndata['hit_type']==1]

tensor([ 1.,  3.,  6.,  8., 10.])


tensor([[ 1.9666],
        [12.0627],
        [ 5.7847],
        [45.5192],
        [ 0.7465]])

In [35]:
print(g.ndata['particle_number'][g.ndata['hit_type']==1])
g.ndata['p_hits'][g.ndata['hit_type']==1]

tensor([ 8.,  3.,  6.,  1., 10.])


tensor([[45.3621],
        [12.1049],
        [ 5.7963],
        [ 1.9674],
        [ 0.7439]])

In [30]:

import pandas as pd
import plotly.express as px

mask = (g.ndata['hit_type'] !=-1)    
tidx =  1*(g.ndata['particle_number'][mask].view(-1,1))
#tidx =    1*(g.ndata['hit_link_modified'][mask].view(-1,1))+1
features =  20*(g.ndata['e_hits'][mask].view(-1,1)) +g.ndata["h"][mask][:,-1].view(-1,1)
X = g.ndata["pos_hits_xyz"][mask] #[mask]
data = {
            "X":X[:, 0].view(-1, 1).detach().cpu().numpy(),
            "Y": X[:, 1].view(-1, 1).detach().cpu().numpy(),
            "Z": X[:, 2].view(-1, 1).detach().cpu().numpy(),
            "tIdx": tidx.view(-1, 1).detach().cpu().numpy(),
            "features": features.view(-1, 1).detach().cpu().numpy(),
        }
hoverdict = {}
# if hoverfeat is not None:
#     for j in range(hoverfeat.shape[1]):
#         hoverdict["f_" + str(j)] = hoverfeat[:, j : j + 1]
#     data.update(hoverdict)

# if nidx is not None:
#     data.update({"av_same": av_same})

df = pd.DataFrame(
np.concatenate([data[k] for k in data], axis=1),
columns=[k for k in data],
)
rdst = np.random.RandomState(1234567890)  # all the same
# shuffle_truth_colors(df, "tIdx", rdst)

hover_data = ["tIdx"] #+ [k for k in hoverdict.keys()]
# if nidx is not None:
#     hover_data.append("av_same")
fig = px.scatter_3d(
df,
x="X",
y="Y",
z="Z",
color="tIdx",
size="features",
hover_data=hover_data,
template="simple_white",
)
fig.update_traces(marker=dict(line=dict(width=0)))

tensor(28.)

In [48]:
(25-8)/25

0.68

In [ ]:
from torch_scatter import scatter_add, scatter_sum, scatter_mean
mask_hit_type_t1 = g.ndata["hit_type"]==2
mask_hit_type_t2 = g.ndata["hit_type"]==1
mask_all = mask_hit_type_t1
mean_pos_cluster = scatter_mean(g.ndata["pos_hits_xyz"][mask_all], g.ndata["particle_number"][mask_all].long().view(-1), dim=0)
mean_pos_cluster
pos_track = g.ndata["pos_hits_xyz"][mask_hit_type_t2]
particle_track = g.ndata["particle_number"][mask_hit_type_t2]
print("particle_track", particle_track)
distance_track_cluster = torch.norm(mean_pos_cluster[particle_track.long()]-pos_track,dim=1)/1000
print("distance_track_cluster", distance_track_cluster)
bad_tracks = distance_track_cluster>0.25
index_bad_tracks = mask_hit_type_t2.nonzero().view(-1)[bad_tracks]
g.ndata["particle_number"][index_bad_tracks]= 0 

In [ ]:
from torch_scatter import scatter_max, scatter_add, scatter_mean
scatter_add(torch.ones_like(g.ndata['particle_number']), g.ndata['particle_number'].long())

In [ ]:
torch.atan(y.coord[32][2]/y.coord[32][1])*180/np.pi

In [ ]:
import pandas as pd
import plotly.express as px
from src.logger.plotting_tools import shuffle_truth_colors
mask =  (g.ndata['particle_number'] ==2) #+ (g.ndata['particle_number'] ==2)
tidx =  g.ndata['particle_number'][mask].view(-1,1) #[mask].view(-1,1) #(labels_pandora.view(-1,1))
features = torch.ones_like(g.ndata["pos_hits_xyz"][mask][:,0]).view(-1,1)
X =g.ndata["conformal"][mask] #[mask]
data = {
            "X":X[:, 0].view(-1, 1).detach().cpu().numpy(),
            "Y": X[:, 1].view(-1, 1).detach().cpu().numpy(),
            "Z": X[:, 2].view(-1, 1).detach().cpu().numpy(),
            "tIdx": tidx.view(-1, 1).detach().cpu().numpy(),
            "features": features.view(-1, 1).detach().cpu().numpy(),
        }
hoverdict = {}
# if hoverfeat is not None:
#     for j in range(hoverfeat.shape[1]):
#         hoverdict["f_" + str(j)] = hoverfeat[:, j : j + 1]
#     data.update(hoverdict)

# if nidx is not None:
#     data.update({"av_same": av_same})

df = pd.DataFrame(
np.concatenate([data[k] for k in data], axis=1),
columns=[k for k in data],
)
rdst = np.random.RandomState(1234567890)  # all the same
# shuffle_truth_colors(df, "tIdx", rdst)

hover_data = ["tIdx"] #+ [k for k in hoverdict.keys()]
# if nidx is not None:
#     hover_data.append("av_same")
fig = px.scatter_3d(
df,
x="X",
y="Y",
z="Z",
color="tIdx",
size="features",
hover_data=hover_data,
template="plotly_dark",
color_continuous_scale=px.colors.sequential.Rainbow,
)
fig.update_traces(marker=dict(line=dict(width=0)))

In [ ]:
y[:,5][19]

In [ ]:
print(diff_x)
print(diff_x[17-1])


In [ ]:
print(diff_z)
print(diff_z[17-1])


In [ ]:
1600,1700

In [ ]:
# import pandas as pd
# import plotly.express as px
# from src.logger.plotting_tools import shuffle_truth_colors
# tidx =  g.ndata['pandora_pfo'][mask].view(-1,1) #(labels_pandora.view(-1,1))
# features = g.ndata["e_hits"][mask]
# X =g.ndata["pos_hits_xyz"][mask]
# data = {
#             "X":X[:, 0].view(-1, 1).detach().cpu().numpy(),
#             "Y": X[:, 1].view(-1, 1).detach().cpu().numpy(),
#             "Z": X[:, 2].view(-1, 1).detach().cpu().numpy(),
#             "tIdx": tidx.view(-1, 1).detach().cpu().numpy(),
#             "features": features.view(-1, 1).detach().cpu().numpy(),
#         }
# hoverdict = {}
# # if hoverfeat is not None:
# #     for j in range(hoverfeat.shape[1]):
# #         hoverdict["f_" + str(j)] = hoverfeat[:, j : j + 1]
# #     data.update(hoverdict)

# # if nidx is not None:
# #     data.update({"av_same": av_same})

# df = pd.DataFrame(
# np.concatenate([data[k] for k in data], axis=1),
# columns=[k for k in data],
# )
# rdst = np.random.RandomState(1234567890)  # all the same
# # shuffle_truth_colors(df, "tIdx", rdst)

# hover_data = ["tIdx"] #+ [k for k in hoverdict.keys()]
# # if nidx is not None:
# #     hover_data.append("av_same")
# fig = px.scatter_3d(
# df,
# x="X",
# y="Y",
# z="Z",
# color="tIdx",
# size="features",
# hover_data=hover_data,
# template="plotly_dark",
# color_continuous_scale=px.colors.sequential.Rainbow,
# )
# fig.update_traces(marker=dict(line=dict(width=0)))

In [ ]:
labels_pandora = g.ndata["pandora_cluster"].long()
labels_pandora = labels_pandora+1
map_from = list(np.unique(labels_pandora))
cluster_id = map(lambda x: map_from.index(x), labels_pandora)
labels_pandora = torch.Tensor(list(cluster_id)).long()

In [ ]:
from src.logger.plotting_tools import shuffle_truth_colors
import pandas as pd
import plotly.express as px
tidx = g.ndata["particle_number"]
X = g.ndata["pos_hits_xyz"]
Edep = torch.ones_like(X[:,0]).view(-1,1)
data = {
    "X": X[:, 0].view(-1, 1).detach().cpu().numpy(),
    "Y": X[:, 1].view(-1, 1).detach().cpu().numpy(),
    "Z": X[:, 2].view(-1, 1).detach().cpu().numpy(),
    "tIdx": tidx.view(-1, 1).detach().cpu().numpy(),
    "features": Edep.view(-1, 1).detach().cpu().numpy(),
}
hoverdict = {}
# if hoverfeat is not None:
#     for j in range(hoverfeat.shape[1]):
#         hoverdict["f_" + str(j)] = hoverfeat[:, j : j + 1]
#     data.update(hoverdict)

# if nidx is not None:
#     data.update({"av_same": av_same})

df = pd.DataFrame(
    np.concatenate([data[k] for k in data], axis=1),
    columns=[k for k in data],
)
df["orig_tIdx"] = df["tIdx"]
# rdst = np.random.RandomState(1234567890)  # all the same
# shuffle_truth_colors(df, "tIdx", rdst)

# hover_data = ["orig_tIdx", "idx"] + [k for k in hoverdict.keys()]
# if nidx is not None:
#     hover_data.append("av_same")
fig = px.scatter_3d(
    df,
    x="X",
    y="Y",
    z="Z",
    color="tIdx",
    size = "features",
    # hover_data=hover_data,
    template="plotly_dark",
    color_continuous_scale=px.colors.sequential.Rainbow,
)
fig.update_traces(marker=dict(line=dict(width=0)))
# path= "/eos/user/m/mgarciam/EVAL_REPOS/Tracking_wcoc/data/coord_n.html"
# fig.write_html(path)

In [ ]:
12,26

In [ ]:
([26, 30, 29,  2,  8, 25, 23, 12, 17, 28, 27, 24, 15,  1, 11,  9, 31, 14,
         6, 10, 
         
         
         
          4, 20,  3, 16,  5, 18,  0, 22, 21,  7, 13, 19]))

In [ ]:
from src.logger.plotting_tools import shuffle_truth_colors
import pandas as pd
import plotly.express as px
tidx = g.ndata["particle_number"]
X = g.ndata["h"][:,0:3]
Edep = torch.ones_like(X[:,0]).view(-1,1)*0.5
data = {
    "X": X[:, 0].view(-1, 1).detach().cpu().numpy(),
    "Y": X[:, 1].view(-1, 1).detach().cpu().numpy(),
    "Z": X[:, 2].view(-1, 1).detach().cpu().numpy(),
    "tIdx": tidx.view(-1, 1).detach().cpu().numpy(),
    "features": Edep.view(-1, 1).detach().cpu().numpy(),
}
hoverdict = {}
# if hoverfeat is not None:
#     for j in range(hoverfeat.shape[1]):
#         hoverdict["f_" + str(j)] = hoverfeat[:, j : j + 1]
#     data.update(hoverdict)

# if nidx is not None:
#     data.update({"av_same": av_same})

df = pd.DataFrame(
    np.concatenate([data[k] for k in data], axis=1),
    columns=[k for k in data],
)
df["orig_tIdx"] = df["tIdx"]
# rdst = np.random.RandomState(1234567890)  # all the same
# shuffle_truth_colors(df, "tIdx", rdst)

# hover_data = ["orig_tIdx", "idx"] + [k for k in hoverdict.keys()]
# if nidx is not None:
#     hover_data.append("av_same")
fig = px.scatter_3d(
    df,
    x="X",
    y="Y",
    z="Z",
    color="tIdx",
    size = "features",
    # hover_data=hover_data,
    template="plotly_dark",
    color_continuous_scale=px.colors.sequential.Rainbow,
)
fig.update_traces(marker=dict(line=dict(width=0)))

In [ ]:
path= "/eos/user/m/mgarciam/EVAL_REPOS/Tracking_wcoc/data/coord.html"
fig.write_html(path)

In [ ]:
from src.models.gravnet_3_L import GravnetModel
load_model_weights = "/eos/user/m/mgarciam/datasets_mlpf/models_trained_CLD/test_L_tracking9/_epoch=30.ckpt" 
model = GravnetModel.load_from_checkpoint(
load_model_weights, args=args, dev=0
)